![](https://blogdozouza.files.wordpress.com/2020/09/profissao-corretor-de-imoveis.jpg)

# Prevendo valores de imóveis

## Problema

Um executivo do ramo imobiliário em Boston-EUA, solicitou a sua equipe de Ciêntistas de Dados que fizessem um modelo para previsão de preços de imóveis com bases em algumas características dos imóveis e de sua localização, características estas tais como: número de quartos, distância entre grande centros, impostos, idade, taxa de crime e etc.

Abaixo, um passo a passa da análise realizada, modelo criado e uma aplicação para uso do executivo.

## Conhecendo a base de dados
- Iremos começar conhecendo melhor a base de dados disponibilizada pela equipe de Engenheiros de Dados...
- E montaremos uma *baseline*.

In [ ]:
# Montar o drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importando as bibliotecas básicas necessárias

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Carregando a base de dados (`boston`)

In [ ]:
from sklearn.datasets import load_boston
boston = load_boston()

Descrição da base de dados: `boston`, aqui podemos ver algumas características da base, tais como:
- número de registros
- números de atributos
- informações sobre esses atributos
- e referências

In [ ]:
print (boston.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

Criando um *dataframe* (data) em *Pandas* para iniciarmos a fase de **Análise Exploratória de Dados** (EDA).

In [ ]:
data = pd.DataFrame(boston.data, columns=boston.feature_names)

Adicionando a coluna (**MEDV**) que será nossa `variável alvo` no *dataframe* **data**.

In [ ]:
data['MEDV'] = boston.target

Vamos ver mais detalhes dos nossos dados (aqui também podemos ver o `tipo de dados`, `memória alocada`)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


Vamos agora converter a coluna número de quartos (RM) para inteiro.

In [ ]:
data.RM = data.RM.astype(int)

Visualizando novamente os dados...

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    int64  
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(13), int64(1)
memory usage: 55.5 KB


Vamos visualizar abaixo uma `Estatística Descritiva` da Coluna Número de Quartos (RM)
- podemos ver abaixo a média(*mean*), mediana(50%), desvio padrão(std) e os quartis.
  - vemos que em média as casas (contidas na base de dados) tem entre 5 e 6 quartos.

In [ ]:
data.RM.describe()

count    506.000000
mean       5.776680
std        0.778123
min        3.000000
25%        5.000000
50%        6.000000
75%        6.000000
max        8.000000
Name: RM, dtype: float64

### Definindo um *Baseline*

- Uma *baseline* é importante para ter marcos no projeto.
- Permite uma explicação fácil para todos os envolvidos.
- É algo que sempre tentaremos ganhar na medida do possível.

A *baseline* que iremos criar é por categoria, ou seja, se o imóvel é Pequeno, Médio ou Grande tendo como base o número de quartos.

Regras:
- Menor ou igual a 4 quartos - **Pequeno**
- Maior que 4 e menor que 7 quartos - **Médio** 
- Maior ou igual a 7 quartos - **Grande**

In [ ]:
# Criando uma variável categorias
categorias = []

In [ ]:
# Alimenta a lista categorias com base nas regras descritas acima
for i in data.RM.iteritems():
  valor = (i[1])
  if valor <= 4:
    categorias.append('Pequeno')
  elif valor < 7:
    categorias.append('Medio')
  else:
    categorias.append('Grande')

Criando uma coluna `categorias` na nossa base de dados (data) com base nas regras do *baseline*.

In [ ]:
data['categorias'] = categorias

Visualizando a quantidade de imóveis por **categoria**.

In [ ]:
# Podemos observar que a grande maioria dos imóveis são médios
data.categorias.value_counts()

Medio      427
Grande      64
Pequeno     15
Name: categorias, dtype: int64

Agora, podemos visualizar o Valor Médio dos imóveis por Categorias (medias_categorias)

In [ ]:
# criando a variável
medias_categorias = data.groupby(by='categorias')['MEDV'].mean()

In [ ]:
# visualizando a variável
medias_categorias

categorias
Grande     38.396875
Medio      20.340281
Pequeno    17.260000
Name: MEDV, dtype: float64

Criaremos um Dicionário para armazenarmos nossa base linha (`chave:valor`)

In [ ]:
# Criando o dicionário com as chaves: médio, grande e pequeno e seus respectivos valores
dic_baseline = {'Grande': medias_categorias[0], 'Medio': medias_categorias[1], 'Pequeno': medias_categorias[2]}

In [ ]:
# Imprime dicionário
dic_baseline

{'Grande': 38.396875, 'Medio': 20.340281030444984, 'Pequeno': 17.26}

Agora, vamos ao principal da *baseline*, iremos criar uma `função` para **retornar o valor médio** com base no número de quartos (o número de quartos será a entrada da função)

In [ ]:
def retorna_baseline(num_quartos):
  if num_quartos <= 4:
    return dic_baseline.get('Pequeno')
  elif num_quartos < 7:
    return dic_baseline.get('Medio')
  else:
    return dic_baseline.get('Grande')

In [ ]:
# Chama a função retorna_baseline (um teste)
retorna_baseline(7)

38.396875

Agora executaremos a função para todos os registros (variável: RM) da base de dados (coleção) para verificar a função...

In [ ]:
for i in data.RM.iteritems():
  n_quartos = i[1]
  print('Número de quartos é: {} , Valor médio: {}'.format(n_quartos,retorna_baseline(n_quartos)))

## Preparação dos Dados para aplicação do *Baseline* e dos algoritmos de *Machine Learning*

Vamos analisar nossa base de dados (coleção), visualizando suas cinco primeiras linhas (`head`)

In [ ]:
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV,categorias
0,0.00632,18.0,2.31,0.0,0.538,6,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,Medio
1,0.02731,0.0,7.07,0.0,0.469,6,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,Medio
2,0.02729,0.0,7.07,0.0,0.469,7,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,Grande
3,0.03237,0.0,2.18,0.0,0.458,6,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,Medio
4,0.06905,0.0,2.18,0.0,0.458,7,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,Grande


**Preparando os dados**

Para criarmos nosso `X` (variáveis independentes) Iremos remover algumas das variáveis ([`colineares`](http://www.portalaction.com.br/analise-de-regressao/36-analise-de-colinearidade-e-multicolinearidade)), a variável: `categorias` e a variável target: `MEDV`.

In [ ]:
# Criando o X
X = data.drop(['RAD','TAX','MEDV','DIS','AGE','ZN','categorias'], axis=1)

Criaremos o `y` (nossa variável `target` - o que pretendemos prever)

In [ ]:
# Variável 'y' é nossa variável alvo (Target)
y = data['MEDV']

Visualizando as primeiras linhas das nossas variáveis `X` e `y`

In [ ]:
# Dados na variável X
X.head()

,CRIM,INDUS,CHAS,NOX,RM,PTRATIO,B,LSTAT
0,0.00632,2.31,0.0,0.538,6,15.3,396.90,4.98
1,0.02731,7.07,0.0,0.469,6,17.8,396.90,9.14
2,0.02729,7.07,0.0,0.469,7,17.8,392.83,4.03
3,0.03237,2.18,0.0,0.458,6,18.7,394.63,2.94
4,0.06905,2.18,0.0,0.458,7,18.7,396.90,5.33


In [ ]:
# Variável alvo
y.head()

0    24.0
1    21.6
2    34.7
3    33.4
4    36.2
Name: MEDV, dtype: float64

Agora, um passo muito importante também, que é separando os conjuntos de `treino` e `teste`, com base no `X` e `y` criados acima.

In [ ]:
# Carrega o método train_test_split
from sklearn.model_selection import train_test_split

In [ ]:
# divide os dados entre o conjunto de treino e teste, 80% e 20% respectivamente.
# define um random_state - quando utilizar esse valor em qualquer lugar, retornará os mesmos dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=5)

In [ ]:
# Visualizando a quantidade de linhas dos conjuntos criados acima
print ('X_train: numero de linhas e colunas: {}'.format(X_train.shape))
print ('X_test: numero de linhas e colunas: {}'.format(X_test.shape))
print ('y_train: numero de linhas e colunas: {}'.format(y_train.shape))
print ('y_test: numero de linhas e colunas: {}'.format(y_test.shape))

X_train: numero de linhas e colunas: (404, 8)
X_test: numero de linhas e colunas: (102, 8)
y_train: numero de linhas e colunas: (404,)
y_test: numero de linhas e colunas: (102,)


## Aplicando o Modelo de *Baseline*
- `Aplicando apenas a média das categorias criadas com base nos números de quartos (explicadas anteriormente)`

Visualizando as cinco primeiras linhas dos dados de teste (`X_test`)

In [ ]:
X_test.head()

,CRIM,INDUS,CHAS,NOX,RM,PTRATIO,B,LSTAT
226,0.38214,6.20,0.0,0.504,8,17.4,387.38,3.13
292,0.03615,4.95,0.0,0.411,6,19.2,396.90,4.70
90,0.04684,3.41,0.0,0.489,6,17.8,392.18,8.81
373,11.10810,18.10,0.0,0.668,4,20.2,396.90,34.77
273,0.22188,6.96,1.0,0.464,7,18.6,390.77,6.58


Criando uma variável (`predicoes`: uma lista de predições), que irá armazenar a predição, com base na *baseline* (variável: RM - número de quartos), para cada um dos registros de teste (`X_test`).

In [ ]:
predicoes = []

In [ ]:
# Para cada registro do teste (X_test) alimenta a Lista Predições
for i in X_test.RM.iteritems():
  n_quartos = i[1]
  predicoes.append(retorna_baseline(n_quartos))

In [ ]:
# Visualizando as 10 primeiras linhas da Lista de Predições
predicoes[:10]

[38.396875,
 20.340281030444984,
 20.340281030444984,
 17.26,
 38.396875,
 20.340281030444984,
 20.340281030444984,
 20.340281030444984,
 20.340281030444984,
 20.340281030444984]

### Log dos Modelos Gerados
É uma boa prática e com isso, mantemos um histórico das nossas análises e sua evolução.

Atribuindo resultados a um *dataframe*

-   `Manteremos um log ao longo do projeto`
-   `Permite a evolução dado o baseline`



In [ ]:
# Cria um dataframe (df_results) vazio
df_results = pd.DataFrame()

Adicionando ao dataframe (df_results) o valor real do `y_test` (coluna: `valor_real`) 

In [ ]:
df_results['valor_real'] = y_test.values

Adicionando ao dataframe (df_results) o valor predito pelo *baseline* (coluna: `valor_predito_baseline`)

In [ ]:
df_results['valor_predito_baseline'] = predicoes

Visualizando as 10 primeiras linhas do df_results (até agora com o valor original e o valor previsto pela *baseline*)

In [ ]:
df_results.head(10)

,valor_real,valor_predito_baseline
0,37.6,38.396875
1,27.9,20.340281
2,22.6,20.340281
3,13.8,17.260000
4,35.2,38.396875
5,10.4,20.340281
6,23.9,20.340281
7,29.0,20.340281
8,22.8,20.340281
9,23.2,20.340281


Vamos Plotar o `valor real` e o `baseline` para termos uma ideia de como ficaram essas predições do *baseline*.

In [ ]:
import plotly.graph_objects as go

# Create traces
fig = go.Figure()

# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_real,
                         mode='lines+markers',
                         name='Valor Real'))

# Linha com os dados preditos
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_baseline,
                         mode='lines+markers',
                         name='Valor Predito Baseline'))
# Plota a figura
fig.show()

print('Podemos observar no plot acima que existe uma grande diferença entre o valor Real e o valor Predito pelo baseline.')
print('Vamos tentar melhorar isso com outros algoritmos de machine learning.')

Podemos observar no plot acima que existe uma grande diferença entre o valor Real e o valor Predito pelo baseline.
Vamos tentar melhorar isso com outros algoritmos de machine learning.


Avaliação do modelo *baseline*, iremos utilizar o RMSE (*ROOT MEAN SQUARE ERROR*).

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
rmse = (np.sqrt(mean_squared_error(y_test, predicoes)))

In [ ]:
print('Performance do modelo baseline:')
print('\nRMSE é: {} '.format(rmse))

# Quanto menor o valor do RMSE, melhor!

Performance do modelo baseline:

RMSE é: 6.327165389325236 


## Aplicando Regressão Linear (Machine Learning)
- `Devido a correlação entre algumas features podemos usar uma regressão linear.`
- `Modelo simples e eficiente.`
- `Solução rápida e robusta.`

In [ ]:
# Carregando o pacote LinearRegression
from sklearn.linear_model import LinearRegression

In [ ]:
# Criando o objeto do tipo LinearRegression
lin_model = LinearRegression()

Treinando o algoritmo e criando o Modelo de `Regressão Linear`

In [ ]:
lin_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
# avaliação do Modelo (lin_model) nos dados de teste (X_test)
y_pred = lin_model.predict(X_test)

In [ ]:
# Calculando a métrica RMSE
rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))

In [ ]:
# Visualizando a Performance do modelo utilizando Regressão Linear
print('Performance do modelo (Regressão Linear) avaliado com os dados de teste:')
print('\nRMSE é: {} '.format(rmse))

print('Podemos observar que já diminuiu em relação a baseline.') 

Performance do modelo (Regressão Linear) avaliado com os dados de teste:

RMSE é: 5.13949386166313 
Podemos observar que já diminuiu em relação a baseline.


In [ ]:
# Atribuindo os resultados da predição (Regressão Linear) ao dataframe df_results
df_results['valor_predito_reg_linear'] = lin_model.predict(X_test)

In [ ]:
df_results.head(10)

,valor_real,valor_predito_baseline,valor_predito_reg_linear
0,37.6,38.396875,36.008112
1,27.9,20.340281,27.776339
2,22.6,20.340281,26.232093
3,13.8,17.260000,1.189862
4,35.2,38.396875,33.437131
5,10.4,20.340281,6.608536
6,23.9,20.340281,25.733199
7,29.0,20.340281,30.466396
8,22.8,20.340281,25.542185
9,23.2,20.340281,20.638996


Aqui já observamos que os valores preditos através da Regressão Linear estão bem mais próximos aos valores reais.

Vamos plotar as três informações para visualizarmos isso melhor
- esse plot é bem iterativo, se quiser comparar apenas duas análises por exemplo, basta desmarcar na legenda a opção que quer tirar da análise...

In [ ]:
import plotly.graph_objects as go

# Create traces
fig = go.Figure()

# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_real,
                         mode='lines+markers',
                         name='Valor Real'))

# Linha com os dados de baseline
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_baseline,
                         mode='lines+markers',
                         name='Baseline'))

# Linha com os dados preditos pela regressão linear
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_reg_linear,
                         mode='lines',
                         line = dict(color = '#FEBFB3'),
                         name='Valor Predito Regressão Linear'))

# Plota a figura
fig.show()

## Aplicando Árvore de Decisão (Machine Learning)
*   `Algoritmo de aprendizado supervisionado`
*   `Explicabilidade do Modelo`
*   `Simples`


In [ ]:
# Importa o pacote DecisionTreeRegressor do skLearn
from sklearn.tree import DecisionTreeRegressor

In [ ]:
# Criando um objeto do tipo DecisionTreeRegressor
regressor = DecisionTreeRegressor()

Treina o Algoritmo e cria o Modelo, utilizando `Decision Tree` (parâmetros *default*)

In [ ]:
regressor.fit(X_train, y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

Avaliando o modelo de Árvore de Decisão

In [ ]:
# Fazendo as previsões com base nos dados de teste (X_test)
y_pred = regressor.predict(X_test)

In [ ]:
# Adicionando o valor do Modelo (Decision Tree) no dataframe df_results (coluna: valor_predito_arvore)
df_results['valor_predito_arvore'] = y_pred

In [ ]:
# Visualizando o dataframe df_results
df_results.head(10)

,valor_real,valor_predito_baseline,valor_predito_reg_linear,valor_predito_arvore
0,37.6,38.396875,36.008112,48.3
1,27.9,20.340281,27.776339,28.5
2,22.6,20.340281,26.232093,22.0
3,13.8,17.260000,1.189862,13.8
4,35.2,38.396875,33.437131,33.2
5,10.4,20.340281,6.608536,8.8
6,23.9,20.340281,25.733199,26.6
7,29.0,20.340281,30.466396,25.0
8,22.8,20.340281,25.542185,18.5
9,23.2,20.340281,20.638996,16.8


Avaliação do Modelo (`Decision Tree`)

In [ ]:
# Calculando a métrica RMSE para as predições do Modelo Decision Tree
rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))

# imprime a performance do modelo
print ('Performance do modelo avaliado com os dados de teste:')
print('\nRMSE é: {} '.format(rmse))

Performance do modelo avaliado com os dados de teste:

RMSE é: 4.534756841093372 


Os resultados melhoram ainda mais, como mostra a métrica RMSE, vamos ver isso graficamente...

In [ ]:
import plotly.graph_objects as go

# cria uma figura
fig = go.Figure()

# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_real,
                         mode='lines+markers',
                         name='Valor Real'))

# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_baseline,
                         mode='lines+markers',
                         name='Valor Predito Baseline'))


# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_reg_linear,
                         mode='lines+markers',
                         name='Valor Predito Reg Liner'))

# Linha com os dados preditos
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_arvore,
                         mode='lines+markers',
                         name='Valor Predito Arvore'))
# Plota a figura
fig.show()

## Aplicando Random Forest (Machine Learning)

- `Método Ensemble`
- `Mais robustez para os modelos`
- `Menos overfitting`

In [ ]:
# Importa o método RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Cria o objeto rf_regressor com parâmetros default
rf_regressor = RandomForestRegressor()

Treinando o algoritmo e gerando o Modelo de `Random Forest`

In [ ]:
rf_regressor.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
# Predições do Modelo (Random Forest) nos dados de teste
y_pred = rf_regressor.predict(X_test)

In [ ]:
# Adicionando os resultados da predição acima no dataframe df_results
df_results['valor_predito_random_forest'] = rf_regressor.predict(X_test)

In [ ]:
# Calculando a métrica RMSE
rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))

In [ ]:
# Visualizando a performance do modelo Random Forest
print ('Performance do Modelo (Random Forest) avaliado com os dados de teste:')
print('\nRMSE é: {} '.format(rmse))

Performance do Modelo (Random Forest) avaliado com os dados de teste:

RMSE é: 3.6953174768755614 


Observamos que melhorou ainda mais o Modelo com base no RMSE, vamos ver agora graficamente...

In [ ]:
# Plotando os resultados dos modelos e o valor real.
import plotly.graph_objects as go

# Create traces
fig = go.Figure()

# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_real,
                         mode='markers',
                         name='Valor Real'))

# Linha com os dados da baseline
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_baseline,
                         mode='lines+markers',
                         line = dict(color = '#FF00FF'),
                         name='Valor da Baseline'))

# Linha com os dados preditos pela arvore
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_arvore,
                         mode='lines',
                         line = dict(color = '#B2FF66'),
                         name='Valor Predito Árvore'))

# Linha com os dados preditos pela regressão linear
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_reg_linear,
                         mode='lines',
                         line = dict(color = '#17BECF'),
                         name='Valor Predito Regressão Linear'))


# Linha com os dados preditos pela Random Forest
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_random_forest,
                         mode='lines',
                         line = dict(color = '#7F7F7F'),
                         name='Valor Predito Random Forest'))

# Plota a figura
fig.show()

## Sugestões de Tunning do Algoritmo de Machine Learning

**Parâmetros da Random Forest**

- `O parâmetro n_estimators define a quantidade de árvores que serão usadas para criação dos modelos. Quanto maior o número de árvores, maior será o número de modelos no ensemble.`

- `O parâmtro max_deph define a profundidade da árvore, quanto mais profunda a árvore, mais especializado nos dados de treino o nosso modelo estará.`

- `O parâmetro max_features define a quantidade máxima de nós que serão usados para separar os níveis folha da árvore`

## Deploy da Aplicação
- Para deploy da aplicação utilizamos o `streamlit`.

In [ ]:
X['MEDV'] = y

In [ ]:
X.head()

,CRIM,INDUS,CHAS,NOX,RM,PTRATIO,B,LSTAT,MEDV
0,0.00632,2.31,0.0,0.538,6,15.3,396.90,4.98,24.0
1,0.02731,7.07,0.0,0.469,6,17.8,396.90,9.14,21.6
2,0.02729,7.07,0.0,0.469,7,17.8,392.83,4.03,34.7
3,0.03237,2.18,0.0,0.458,6,18.7,394.63,2.94,33.4
4,0.06905,2.18,0.0,0.458,7,18.7,396.90,5.33,36.2


In [ ]:
X.to_csv('data.csv', index=False)

### Como colocar a Aplicação em Produção
- Para deploy da aplicação utilizamos o `streamlit`.
  - pré-requisitos - `streamlit`, `plotly`, `plotly-express` e `sklearn` (maiores informações no arquivo `log_instalacao.txt`)
- Passo a passo
  - baixe os arquivos (dentro da pasta: `Deploy`)
    - pasta `model`
      - `data.csv`
    - arquivo `app.py`
      - no diretório do app.py, no prompt de comando execute
        - `streamlit run app.py`

Visual da aplicação
![](https://blogdozouza.files.wordpress.com/2020/09/zoua.png)

## Referências
- Esse material teve como base a Semana de Data Science do Canal Minerando Dados (Parabéns Minerando Dados pelo excelente conteúdo disponibilizado)